# SIGNAL Statement

**Applies to:** Databricks SQL, Databricks Runtime 16.3+

Raises a condition. Use inside a compound statement.

Syntax: `SIGNAL condition_name [SET MESSAGE_TEXT = ... | MESSAGE_ARGUMENTS = map]` or `SIGNAL SQLSTATE value [SET MESSAGE_TEXT = ...]`.

**Note:** From within a handler, prefer **RESIGNAL** to preserve the diagnostic stack; SIGNAL clears it.

In [0]:
USE CATALOG healthcare_dev;
USE SCHEMA gold;

## Example: Validate state filter and SIGNAL if invalid (using workshop logic)

In [0]:
BEGIN
  DECLARE state_filter STRING DEFAULT 'XX';
  DECLARE valid INT DEFAULT (SELECT COUNT(*) FROM dim_cms_beneficiary WHERE sp_state_code = state_filter);
  IF valid = 0 AND state_filter IS NOT NULL THEN
    SIGNAL SQLSTATE '45000' SET MESSAGE_TEXT = 'No beneficiaries found for the given state.';
  END IF;
  SELECT state_filter AS state, valid AS beneficiary_count;
END

## Example: SIGNAL USER_RAISED_EXCEPTION with MESSAGE_ARGUMENTS (parameterized message)

In [0]:
BEGIN
  DECLARE claim_cnt BIGINT DEFAULT (SELECT COUNT(*) FROM fact_cms_outpatient_claims) + (SELECT COUNT(*) FROM fact_cms_inpatient_claims);
  DECLARE arg_map MAP<STRING, STRING>;
  IF claim_cnt = 0 THEN
    SET arg_map = map('errorMessage', 'No claims in gold fact tables.');
    SIGNAL USER_RAISED_EXCEPTION SET MESSAGE_ARGUMENTS = arg_map;
  END IF;
  SELECT claim_cnt AS count;
END